# Cricket Query AI - Main System
## Author: Charan Kumar Pathakamuri
## Project: Cricket Query AI
  

## 1. Overview\n",
    "This notebook serves as the main execution engine for the Cricket Query AI. It integrates the Vector Store (Memory), the Large Language Model (Brain), and the SQL Database to create a RAG-based Text-to-SQL system."
   ]

## 2. Environment Setup and Agent Initialization\n",
    "Here we import the necessary libraries and define the custom `MyVanna` class. This class combines **ChromaDB** (for local vector storage) and **Anthropic's Claude 3 Haiku** (for fast and efficient SQL generation)"

In [12]:
import os
from dotenv import load_dotenv
from vanna.chromadb import ChromaDB_VectorStore
from vanna.anthropic import Anthropic_Chat  # Import the Anthropic component

# Load environment variables
load_dotenv()

# Get the API key from your .env file
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

if not anthropic_api_key:
    raise ValueError("API key not found. Please check your .env file for 'ANTHROPIC_API_KEY'.")

class MyVanna(ChromaDB_VectorStore, Anthropic_Chat):
    def __init__(self, config=None):
        # 1. Memory Configuration (ChromaDB)
        chroma_config = {
            'path': 'E:/Aichatbot/vector_dataBase_CL' # Your vector database path
        }
        
        # 2. Brain Configuration (Claude 3 Haiku)
        anthropic_config = {
            'api_key': anthropic_api_key,
            'model': 'claude-3-haiku-20240307' 
        }
        
        # Initialize both parent classes
        ChromaDB_VectorStore.__init__(self, config=chroma_config)
        Anthropic_Chat.__init__(self, config=anthropic_config)

# Initialize Vanna
# You may need to restart your kernel after the pip install
vn = MyVanna()

# Confirm the setup
print(f"✅ DEBUG: Vanna object created successfully.")
print(f"🧠 Model: {vn.config['model']}")


✅ DEBUG: Vanna object created successfully.
🧠 Model: claude-3-haiku-20240307


## 3. Database Connection\n",
    "We establish a connection to the Microsoft SQL Server using ODBC drivers. This enables the agent to execute the generated SQL queries against the actual `Cricket` database."


In [13]:
conn_str = (
    r"Driver={ODBC Driver 17 for SQL Server};"
    r"Server=CharanKumar\SQLEXPRESS;"
    r"Database=Cricket;"
    r"Trusted_Connection=yes;"
)

vn.connect_to_mssql(odbc_conn_str=conn_str) 


## 4. Model Training
    "To ensure high accuracy, we train the Vanna agent on specific knowledge bases:
    1.  DDL (Data Definition Language): The schema structure of the tables.
    2.  Documentation: Contextual information and business definitions.
    3.  Training Queries: Golden examples of Question-SQL pairs.

In [14]:

from Training import Documentation, schema_text, training_queries

vn.train(ddl=schema_text)
vn.train(documentation=Documentation)
for pair in training_queries:
    vn.train(sql=pair["sql"], question=pair["question"])


Adding ddl: 
CREATE TABLE [dbo].[asiacup](
	[Team] [nvarchar](50) NOT NULL,
	[Opponent] [nvarchar](50) NOT NULL,
	[Format] [nvarchar](50) NOT NULL,
	[Ground] [nvarchar](50) NOT NULL,
	[Year] [smallint] NOT NULL,
	[Toss] [nvarchar](50) NOT NULL,
	[Selection] [nvarchar](50) NOT NULL,
	[Run_Scored] [smallint] NULL,
	[Wicket_Lost] [tinyint] NULL,
	[Fours] [tinyint] NULL,
	[Sixes] [tinyint] NULL,
	[Extras] [tinyint] NULL,
	[Run_Rate] [float] NULL,
	[Avg_Bat_Strike_Rate] [float] NULL,
	[Highest_Score] [tinyint] NULL,
	[Wicket_Taken] [tinyint] NULL,
	[Given_Extras] [tinyint] NULL,
	[Highest_Individual_wicket] [tinyint] NULL,
	[Player_Of_The_Match] [nvarchar](50) NULL,
	[Result] [nvarchar](50) NOT NULL
) ON [PRIMARY]

CREATE TABLE [dbo].[champion](
	[Year] [smallint] NOT NULL,
	[Host] [nvarchar](50) NOT NULL,
	[No_Of_Team] [tinyint] NOT NULL,
	[Champion] [nvarchar](50) NOT NULL,
	[Runner_Up] [nvarchar](50) NOT NULL,
	[Player_Of_The_Series] [nvarchar](50) NOT NULL,
	[Highest_Run_Scorer] [nvarch

## 5. Testing the Agent
    Before full deployment, we perform a test query to verify that the agent can retrieve context, generate valid SQL, and fetch results from the database.

In [15]:
question='List Top 5 Batsmen in terms of runs in ODI'
sql_query = vn.generate_sql(question)
vn.ask(question)

SQL Prompt: [{'role': 'system', 'content': "You are a T-SQL / Microsoft SQL Server expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n===Additional Context \n\n===Response Guidelines \n1. If the provided context is sufficient, please generate a valid SQL query without any explanations for the question. \n2. If the provided context is almost sufficient but requires knowledge of a specific string in a particular column, please generate an intermediate SQL query to find the distinct strings in that column. Prepend the query with a comment saying intermediate_sql \n3. If the provided context is insufficient, please explain why it can't be generated. \n4. Please use the most relevant table(s). \n5. If the question has been asked and answered before, please repeat the answer exactly as it was given before. \n6. Ensure that the output SQL is T-SQ

('SELECT TOP 5 Player_Name, Runs\nFROM dbo.[batsman data odi]\nORDER BY Runs DESC;',
       Player_Name  Runs
 0  ST Jayasuriya   1220
 1  KC Sangakkara   1075
 2   SR Tendulkar    971
 3   Shoaib Malik    786
 4      RG Sharma    745,
 Figure({
     'data': [{'type': 'bar',
               'x': array(['ST Jayasuriya ', 'KC Sangakkara ', 'SR Tendulkar ', 'Shoaib Malik ',
                           'RG Sharma '], dtype=object),
               'y': array([1220, 1075,  971,  786,  745], dtype=int64)}],
     'layout': {'template': '...', 'title': {'text': 'Top 5 Batsmen in terms of runs in ODI'}}
 }))